In [59]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [5]:
!python -V

Python 3.9.13


In [7]:
tf.__version__

'2.9.1'

In [12]:
!wget http://bit.ly/mlbookcamp-pants -O pants.jpg

--2022-11-23 20:38:16--  http://bit.ly/mlbookcamp-pants
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg [following]
--2022-11-23 20:38:17--  https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23048 (23K) [image/jpeg]
Saving to: ‘pants.jpg’

pants.jpg           100%[===================>]  22.51K  --.-KB/s    in 0.002s  

2022-11-23 20:38:17 (9.78 MB/s) - ‘pants.jp

In [62]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [63]:
model = keras.models.load_model('xception_v4_15_0.871.h5')

In [61]:
img = load_img('pants.jpg', target_size=(299,299))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [28]:
X.shape

(1, 299, 299, 3)

In [31]:
preds = model.predict(X)

1/1 [==============================] - 0s 21ms/step


In [32]:
classes = ['dress','hat','longsleeve','outwear','pants','shirt','shoes','shorts','skirt','t-shirt']

In [33]:
dict(zip(classes, preds[0]))

{'dress': -2.6031165,
 'hat': -7.704868,
 'longsleeve': -0.21532209,
 'outwear': -0.2442853,
 'pants': 10.73538,
 'shirt': -1.4546652,
 'shoes': -4.4502616,
 'shorts': 3.4037158,
 'skirt': -3.2864554,
 't-shirt': -6.1116776}

## Convert Keras to TF-Lite

In [38]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('clothing_model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpo243l2qb/assets


INFO:tensorflow:Assets written to: /tmp/tmpo243l2qb/assets
2022-11-23 20:49:53.499392: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-23 20:49:53.499440: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-23 20:49:53.499615: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpo243l2qb
2022-11-23 20:49:53.533031: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-11-23 20:49:53.533075: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpo243l2qb
2022-11-23 20:49:53.660266: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-11-23 20:49:54.233278: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpo243l2qb
2022-11-23 20:49:54.437553: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

In [39]:
import tensorflow.lite as tflite

In [57]:
interpreter = tflite.Interpreter(model_path='clothing_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [58]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [56]:
dict(zip(classes, preds[0]))

{'dress': -2.6031208,
 'hat': -7.704869,
 'longsleeve': -0.21532193,
 'outwear': -0.2442839,
 'pants': 10.735379,
 'shirt': -1.4546633,
 'shoes': -4.450262,
 'shorts': 3.4037201,
 'skirt': -3.2864566,
 't-shirt': -6.11168}

## Removing TF dependency

In [82]:
from PIL import Image

In [83]:
with Image.open('pants.jpg') as f:
    img = img.resize((299,299), Image.Resampling.NEAREST)

In [84]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [85]:
img = load_img('pants.jpg', target_size=(299,299))

x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [86]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [87]:
classes = ['dress','hat','longsleeve','outwear','pants','shirt','shoes','shorts','skirt','t-shirt']

In [89]:
dict(zip(classes, preds[0]))

{'dress': -2.6031208,
 'hat': -7.704869,
 'longsleeve': -0.21532193,
 'outwear': -0.2442839,
 'pants': 10.735379,
 'shirt': -1.4546633,
 'shoes': -4.450262,
 'shorts': 3.4037201,
 'skirt': -3.2864566,
 't-shirt': -6.11168}

## Simple way of doing it

In [2]:
!pip install keras-image-helper

  Using cached keras_image_helper-0.0.1-py3-none-any.whl (4.6 kB)


In [1]:
!pip install tflite-runtime

  Using cached tflite_runtime-2.10.0-cp39-cp39-manylinux2014_x86_64.whl (2.5 MB)


In [1]:
#import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [3]:
interpreter = tflite.Interpreter(model_path='clothing_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [4]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [5]:
url = 'http://bit.ly/mlbookcamp-pants'
X=preprocessor.from_url(url)

In [6]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [7]:
classes = ['dress','hat','longsleeve','outwear','pants','shirt','shoes','shorts','skirt','t-shirt']

In [8]:
dict(zip(classes, preds[0]))

{'dress': -2.6031208,
 'hat': -7.704869,
 'longsleeve': -0.21532193,
 'outwear': -0.2442839,
 'pants': 10.735379,
 'shirt': -1.4546633,
 'shoes': -4.450262,
 'shorts': 3.4037201,
 'skirt': -3.2864566,
 't-shirt': -6.11168}